In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd

import wfdb
import os
import wfdb.processing as wp
import matplotlib.pyplot as plt
from scipy import signal
import shutil

import time
from scipy import sparse
import os
import warnings
import scipy.io as sio

from resnet_ecg.densenet import DenseNet

warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
train_dataset_path = os.getcwd()+"/Train/"
val_dataset_path = os.getcwd()+"/Val/"

train_files = os.listdir(train_dataset_path)
train_files.sort()
val_files = os.listdir(val_dataset_path)
val_files.sort()

In [3]:
labels = pd.read_csv("en_all_labels.csv")
labels.head()

,File_name,label1,label2,label3,label4,label5,label6,label7,label8
0,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN0002,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRAIN0002,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
labels.shape

(19500, 9)

In [5]:
train_x = np.load('en_all_train_x.npy')#en_train_x_seg

In [6]:
train_x.shape

(19500, 3000, 12)

In [7]:
''' 
sum1_label=0
sum2_label=0
sum3_label=0
sum4_label=0
sum5_label=0
sum6_label=0
sum7_label=0
''' 
bin_label = np.zeros((labels.shape[0],9))
for i in range(labels.shape[0]):
    label_nona = labels.loc[i].dropna()
    ''' 
    if label_nona.shape[0] == 2:
        sum1_label+=1
    if label_nona.shape[0] == 3:
        sum2_label+=1
    if label_nona.shape[0] == 4:
        sum3_label+=1
    if label_nona.shape[0] == 5:
        sum4_label+=1
    if label_nona.shape[0] == 6:
        sum5_label+=1
    if label_nona.shape[0] == 7:
        sum6_label+=1
    if label_nona.shape[0] == 8:
        sum7_label+=1
    '''
    for j in range(1,label_nona.shape[0]):
        bin_label[i,int(label_nona[j])]=1

In [8]:
bin_label.shape

(19500, 9)

In [9]:
from keras.utils import to_categorical
from keras.optimizers import SGD,Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,EarlyStopping,ReduceLROnPlateau
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from sklearn.model_selection import StratifiedKFold

In [10]:
class Config(object):
    def __init__(self):
        self.conv_subsample_lengths = [1, 2, 1, 2, 1, 2, 1, 2]
        self.conv_filter_length = 32
        self.conv_num_filters_start = 12
        self.conv_init = "he_normal"
        self.conv_activation = "relu"
        self.conv_dropout = 0.5
        self.conv_num_skip = 2
        self.conv_increase_channels_at = 2
        self.batch_size = 32#128
        self.input_shape = [2560, 12]#[1280, 1]
        self.num_categories = 2

    @staticmethod
    def lr_schedule(epoch):
        lr = 0.1
        if epoch >= 20 and epoch < 40:
            lr = 0.01
        if epoch >= 40:
            lr = 0.001
        print('Learning rate: ', lr)
        return lr

In [11]:
config = tf.ConfigProto()

config.gpu_options.per_process_gpu_memory_fraction = 0.5

session = tf.Session(config=config)

KTF.set_session(session )

n_steps = 3000
model = DenseNet(input_shape=(n_steps,12), nb_classes=9, depth=10, growth_rate=16,
                          dropout_rate=0.1, bottleneck=False, compression=0.5)

Creating DenseNet 0.0.1
#############################################
Dense blocks: 3
Layers per dense block: [2, 2, 2]
#############################################
Instructions for updating:
Colocations handled automatically by placer.
block::: 0
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
block::: 1
nb_channels:: Tensor("concatenate_6/concat:0", shape=(?, 187, 60), dtype=float32)


In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3000, 12)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 3000, 16)     6144        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 3000, 16)     64          conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 3000, 16)     0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv1d_2 (

In [13]:
import keras.backend as K

def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    # Calculates the F score, the weighted harmonic mean of precision and recall.
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
    
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    # Calculates the f-measure, the harmonic mean of precision and recall.
    return fbeta_score(y_true, y_pred, beta=1)


In [14]:
from sklearn.model_selection import train_test_split
X_tr, X_vld, lab_tr, lab_vld = train_test_split(train_x, labels['label1'], 
                                        stratify = labels['label1'], #test_size=0.2,
                                        shuffle=True, random_state = 123)

In [15]:
lab_tr.index.tolist()[:5]

[8083, 15109, 1974, 538, 6050]

In [16]:
def preprocess_y(y,num_class=9):
    bin_label = np.zeros((y.shape[0],num_class))
    for i in range(y.shape[0]):
        label_nona = labels.loc[lab_tr.index.tolist()[i]].dropna()
        for j in range(1,label_nona.shape[0]):
            bin_label[i,int(label_nona[j])]=1
    return bin_label

In [17]:
y_tr = preprocess_y(lab_tr)
y_vld = preprocess_y(lab_vld) 

In [18]:
checkpointer = ModelCheckpoint(filepath='weights_best_simple_model_2.hdf5', 
                            monitor='val_fmeasure',verbose=1, save_best_only=True, mode='max')
reduce = ReduceLROnPlateau(monitor='val_fmeasure',factor=0.5,patience=2,verbose=1,min_delta=1e-4,mode='max')
''' 
model.compile(optimizer = 'adam',
           loss='binary_crossentropy',
           metrics=['accuracy',fmeasure,recall,precision])

epochs = 20

history = model.fit_generator(train_generator,
       validation_data = val_generator,
       epochs=epochs,
       callbacks=[checkpointer,reduce],
       verbose=1)
'''
pass

In [19]:
from keras import backend as K

#2156+224+672+654+180+826+534+504+1953

def weighted_loss(labels,logits):
    logits_log = (-1) * K.log(logits)
    num_class = logits.shape[-1]
    
    y_0 = 0.8 * labels[:,0:1]
    y_1 = 1.2 * labels[:,1:2]
    y_2 = 1.0 * labels[:,2:3]
    y_3 = 1.0 * labels[:,3:4]
    y_4 = 1.2 * labels[:,4:5]
    y_5 = 1.0 * labels[:,5:6]
    y_6 = 1.0 * labels[:,6:7]
    y_7 = 1.0 * labels[:,7:8]
    y_8 = 0.8 * labels[:,8:]
    
    weight_labels = K.concatenate([y_0,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8],axis=1)
    loss = K.sum(logits_log * weight_labels,axis=1)
    
    loss = K.mean(loss)
    
    return loss

In [19]:
def add_compile(model, config):
    
    optimizer = SGD(lr=config.lr_schedule(0), momentum=0.9)#Adam()#
    model.compile(loss='binary_crossentropy',#weighted_loss,#'binary_crossentropy',
                  optimizer='adam',#optimizer
                  metrics=['accuracy',fmeasure,recall,precision])   

In [20]:
config = Config()
add_compile(model, config)

model_name = 'resnet12.h5'
earlystop = EarlyStopping(
            monitor='val_fmeasure',#'val_categorical_accuracy',
            patience=10,
            )
checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor='val_categorical_accuracy', mode='max',
                             save_best_only='True')

lr_scheduler = LearningRateScheduler(config.lr_schedule)

callback_lists = [checkpointer,reduce]#[checkpointer,earlystop,lr_scheduler]
#[checkpoint, earlystop,lr_scheduler] 

history = model.fit(x=X_tr, y=y_tr, batch_size=64, epochs=50,  #class_weight='auto',
          verbose=1, validation_data=(X_vld, y_vld), callbacks=callback_lists )

Learning rate:  0.1
Instructions for updating:
Use tf.cast instead.
Train on 14625 samples, validate on 4875 samples
Epoch 1/50
14625/14625 [==============================] - 27s 2ms/step - loss: 0.3479 - acc: 0.8786 - fmeasure: 0.4139 - recall: 0.3063 - precision: 0.7080 - val_loss: 0.5680 - val_acc: 0.8333 - val_fmeasure: 0.2111 - val_recall: 0.1708 - val_precision: 0.2789

Epoch 00001: val_fmeasure improved from -inf to 0.21109, saving model to weights_best_simple_model_2.hdf5
Epoch 2/50
14625/14625 [==============================] - 18s 1ms/step - loss: 0.2595 - acc: 0.9073 - fmeasure: 0.5174 - recall: 0.3804 - precision: 0.8173 - val_loss: 0.6726 - val_acc: 0.8396 - val_fmeasure: 0.2154 - val_recall: 0.1685 - val_precision: 0.3006

Epoch 00002: val_fmeasure improved from 0.21109 to 0.21539, saving model to weights_best_simple_model_2.hdf5
Epoch 3/50
14625/14625 [==============================] - 18s 1ms/step - loss: 0.2322 - acc: 0.9156 - fmeasure: 0.5773 - recall: 0.4411 - precis

In [21]:
from resnet_ecg.ecg_preprocess import ecg_preprocessing
def read_data_seg(data_path, split = "Val",preprocess=False):
    """ Read data """
    # Fixed params
    n_class = 2
    n_steps = 3000#2560

    # Paths
    path_signals = os.path.join(data_path, split)

    # Read time-series data
    channel_files = os.listdir(path_signals)
    #print(channel_files)
    channel_files.sort()
    n_channels = 12#len(channel_files)
    #posix = len(split) + 5

    # Initiate array
    list_of_channels = []
    X = np.zeros((len(channel_files), n_steps, n_channels))
    i_ch = 0
    
    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1','II', 'aVR', 'V3', 'III', 'V5']
    channel_mid_name = ['II','aVR','V2','V5']
    channel_post_name = ['III','aVF','V3','V6']
    
    for i_ch,fil_ch in tqdm(enumerate(channel_files[:])):
        ecg = sio.loadmat(os.path.join(path_signals,fil_ch))
        ecg_channels = np.zeros((n_steps, n_channels))
        for i_n,ch_name in enumerate(channel_name):
            
            if ch_name in channel_mid_name:
                mid_ind = int(ecg[ch_name].T.shape[0]/2)
                ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[mid_ind-2500:mid_ind+2500],n_steps).T 
            elif ch_name in channel_post_name:
                ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[-5000:],n_steps).T
            else:
                ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[:5000],n_steps).T
            #ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T,2560).T
            if preprocess:
                data = ecg_preprocessing(ecg_channels[:,i_n].reshape(1,n_steps), 'sym8', 8, 3, n_steps/10)
                ecg_channels[:,i_n] = data[0]#ecg['data']
            else:
                pass
        X[i_ch,:,:] = ecg_channels
    return X

In [33]:
from resnet_ecg.ecg_preprocess import ecg_preprocessing
import pywt
def read_data_sample(data_path, split = "Val",preprocess=False):
    """ Read data """
    # Fixed params
    n_class = 2
    n_steps = 3000#2560

    path_signals = os.path.join(data_path, split)

    # Read time-series data
    channel_files = os.listdir(path_signals)
    #print(channel_files)
    channel_files.sort()
    n_channels = 12#len(channel_files)

    # Initiate array
    list_of_channels = []
    X = np.zeros((len(channel_files), n_steps, n_channels))
    i_ch = 0
    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1',  'II', 'aVR', 'V3', 'III', 'V5']
    
    #shape_ecg = []
    
    for i_ch,fil_ch in tqdm(enumerate(channel_files[:])):       
        ecg = sio.loadmat(os.path.join(path_signals,fil_ch))        
        ecg_channels = np.zeros((n_steps, n_channels))        
        for i_n,ch_name in enumerate(channel_name[:]):            
            ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T,n_steps).T
            #shape_ecg.append(ecg[ch_name].shape[1])
            if preprocess:
                data = ecg_preprocessing(ecg_channels[:,i_n].reshape(1,n_steps), 'sym8', 8, 3, n_steps/10)
                ecg_channels[:,i_n] = data[0]#ecg['data']
            else:
                pass
        X[i_ch,:,:] = ecg_channels

    return X#shape_ecg#

In [23]:
test_x = read_data_sample(os.getcwd(),split='Val',preprocess=True)#read_data_sample  read_data_seg

500it [00:08, 58.72it/s]


In [24]:
test_x.shape

(500, 3000, 12)

In [34]:
def wavelet(ecg,wavefunc,lv,m,n):   #
    
    coeff = pywt.wavedec(ecg,wavefunc,mode='sym',level=lv)   #
    #sgn = lambda x: 1 if x > 0 else -1 if x < 0 else 0

    for i in range(m,n+1):  
        cD = coeff[i]
        for j in range(len(cD)):
            Tr = np.sqrt(2*np.log(len(cD)))  
            if cD[j] >= Tr:
                coeff[i][j] = np.sign(cD[j]) - Tr 
            else:
                coeff[i][j] = 0   
                
    denoised_ecg = pywt.waverec(coeff,wavefunc)
    return denoised_ecg

def wavelet_db6(sig):
    """
    R J, Acharya U R, Min L C. ECG beat classification using PCA, LDA, ICA and discrete
     wavelet transform[J].Biomedical Signal Processing and Control, 2013, 8(5): 437-448.

    param sig: 1-D numpy Array
    return: 1-D numpy Array
    """

    coeffs = pywt.wavedec(sig, 'db6', level=9)

    coeffs[-1] = np.zeros(len(coeffs[-1]))

    coeffs[-2] = np.zeros(len(coeffs[-2]))

    coeffs[0] = np.zeros(len(coeffs[0]))

    sig_filt = pywt.waverec(coeffs, 'db6')

    return sig_filt

def read_test_data(data_path, split = "Val",preprocess=True):
    """ Read data """

    # Fixed params
    n_class = 2
    n_steps = 3000 #2560

    # Paths
    path_signals = os.path.join(data_path, split)

    # Read time-series data
    channel_files = os.listdir(path_signals)
    #print(channel_files)
    channel_files.sort()
    n_channels = 12#len(channel_files)

    # Initiate array
    list_of_channels = []
    #X = np.zeros((len(channel_files), n_steps, n_channels))
    
    i_ch = 0
    
    data_x_w1 = []
    data_x_w2 = []
    data_x_w3 = []
    
    data_x = []
    data_y = []
    
    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1','II', 'aVR', 'V3', 'III', 'V5']
    channel_mid_name = ['II','aVR','V2','V5']
    channel_post_name = ['III','aVF','V3','V6']
    
    for i_ch,fil_ch in tqdm(enumerate(channel_files[:])):
        #print(fil_ch)

        ecg = sio.loadmat(os.path.join(path_signals,fil_ch))
        
        if True:#7 in labels_list[1:] or 4 in labels_list[1:]:
            for i_filter in range(3):
                
                ecg_channels_w1 = np.zeros((n_steps, n_channels))
                ecg_channels_w2 = np.zeros((n_steps, n_channels))
                ecg_channels_w3 = np.zeros((n_steps, n_channels))
                ecg_channels    = np.zeros((n_steps, n_channels))
                
                for i_n,ch_name in enumerate(channel_name[:]):

                    # method 1
                    '''  
                    if ch_name in channel_mid_name:
                        mid_ind = int(ecg[ch_name].T.shape[0]/2)
                        ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[mid_ind-2500:mid_ind+2500],n_steps).T 
                    elif ch_name in channel_post_name:
                        ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[-5000:],n_steps).T
                    else:
                        ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[:5000],n_steps).T
                    '''
                    #method 2
                    ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T,n_steps).T
                    
                    #print(ecg_channels[:,i_n].shape)
                    if preprocess and i_filter == 0:
                        data = ecg_preprocessing(ecg_channels[:,i_n].reshape(1,n_steps),'sym8',8,3,n_steps/10)
                        ecg_channels_w1[:,i_n] = data[0]#ecg['data']
                    elif i_filter == 1:
                        ecg_channels_w2[:,i_n] = wavelet(ecg_channels[:,i_n],'db4',4,2,4)[0]
                    elif i_filter == 1:
                        ecg_channels_w3[:,i_n] = wavelet_db6(ecg_channels[:,i_n].reshape(1,n_steps))[0]
                    else:
                        pass
                        #ecg_channels[:,i_n] = ecg_channels[:,i_n]
                #X[i_ch,:,:] = ecg_channels
                data_x_w1.append(ecg_channels_w1)
                data_x_w2.append(ecg_channels_w2)
                data_x_w3.append(ecg_channels_w3)
        else:
            ecg_channels = np.zeros((n_steps, n_channels))
            for i_n,ch_name in enumerate(channel_name[:]):

                # method 1
                '''
                if ch_name in channel_mid_name:
                    mid_ind = int(ecg[ch_name].T.shape[0]/2)
                    ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[mid_ind-2500:mid_ind+2500],2560).T 
                elif ch_name in channel_post_name:
                    ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[-5000:],2560).T
                else:
                    ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[:5000],2560).T
                '''
                #method 2
                ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T,n_steps).T

                if preprocess:
                    data = ecg_preprocessing(ecg_channels[:,i_n].reshape(1,n_steps),'sym8',8,3,n_steps/10)
                    ecg_channels[:,i_n] = data[0]#ecg['data']
                else:
                    pass

            #X[i_ch,:,:] = ecg_channels
            data_x.append(ecg_channels)

    # Return 
    return np.array(data_x_w1).astype('float32'),np.array(data_x_w2).astype('float32'),np.array(data_x_w3).astype('float32')#X

In [50]:
#test_x_w1,test_x_w2,test_x_w3 = read_test_data(os.getcwd())
#test_y_w1 = model.predict(test_x_w1)
#test_y_w2 = model.predict(test_x_w2)
#test_y_w3 = model.predict(test_x_w3)
#test_y = test_y_w1+test_y_w2+test_y_w3

In [46]:
test_y = model.predict(test_x)

In [47]:
test_y

array([[1.63643658e-01, 4.57567513e-01, 4.29272652e-04, ...,
        1.67076588e-02, 1.49106979e-03, 3.85248959e-02],
       [2.24000216e-03, 1.06341064e-01, 2.53799558e-03, ...,
        3.16327214e-02, 4.67002392e-05, 1.14878714e-02],
       [3.58781219e-03, 9.06751096e-01, 2.02680826e-02, ...,
        6.52618408e-02, 5.51044941e-04, 3.27953279e-01],
       ...,
       [5.66244125e-07, 3.39648128e-03, 4.09376621e-03, ...,
        5.45324147e-01, 1.22922659e-03, 3.32450867e-03],
       [2.06321955e-01, 1.28063560e-03, 5.16609073e-01, ...,
        3.33055258e-02, 3.77246737e-03, 5.29712558e-01],
       [2.18667775e-01, 2.04932988e-01, 1.55791640e-03, ...,
        1.11970663e-01, 2.40185857e-03, 7.78462589e-02]], dtype=float32)

In [52]:
y_tr

array([[0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [53]:
x_tr_y = model.predict(X_tr)

In [54]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss

In [55]:
threshold = np.arange(0.1,0.9,0.1)

out = x_tr_y
y_test = y_tr

acc = []
accuracies = []
best_threshold = np.zeros(out.shape[1])
for i in range(out.shape[1]):
    y_prob = np.array(out[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append( matthews_corrcoef(y_test[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

best_threshold

y_pred = np.array([[1 if out[i,j]>=best_threshold[j] else 0 for j in range(y_test.shape[1])] for i in range(len(y_test))])

y_pred 

y_test

hamming_loss(y_test,y_pred)

0.05860588793922127

In [56]:
best_threshold

array([0.7, 0.7, 0.6, 0.6, 0.2, 0.3, 0.4, 0.2, 0.6])

In [97]:
pred = out#np.array([[ 0.01491354,  0.00838183,  0.64092934,  0.99065214,  0.00579561]])
y_pred = []
classes = [0,1,2,3,4,5,6,7,8]
for j in range(pred.shape[0]):
    y_pred.append([1 if pred[j,i]>=best_threshold[i] else 0 for i in range(pred.shape[1])])

In [106]:
pred=[]
for j in range(out.shape[0]):
    pred.append([classes[i] for i in range(9) if y_pred[j][i] == 1])

In [107]:
with open('answers11111.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

In [48]:
import csv
thred = 0.2

pred = []
for i in range(test_y.shape[0]):
    pred.append( list(np.hstack(np.argwhere(test_y[i]>thred))))
    
with open('answers.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

ValueError: need at least one array to concatenate

In [51]:
import csv
thred = 0.15#0.25

pred = []
for i in range(test_y.shape[0]):
    if np.argmax(test_y[i]) == 0:
        pred.append([0])
        continue
    try:
        pred_list = list(np.hstack(np.argwhere(test_y[i]>thred)))
    except ValueError:
        print('valueError')
        pred_list = ['']
        
    if 0 in pred_list:
        if np.argmax(test_y[i]) != 0:
            pred_list.remove(0)
            
    pred.append(pred_list)
    
with open('answers.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()